In [1]:
import folium
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [33]:
geo_path = r'data/antarctic_ice_edge.json'
topo_path1 = r'ch-cantons.topojson.json'
ch_map = folium.Map(location=[46.8418522,7.9253733],
                   tiles='Mapbox Bright', zoom_start=8)
ch_map.choropleth(geo_path=geo_path)
ch_map.choropleth(geo_path=topo_path1,topojson='objects.cantons')
ch_map.save('ch_map.html')

In [147]:
# read the data
p3_grant = r'P3_GrantExport.csv'
grant_data = pd.read_csv(p3_grant,sep = ';')
grant_data = pd.DataFrame(grant_data[~grant_data.University.isnull()])

In [152]:
un_splited_uni_name = grant_data['University'].unique()
uni_name_matrix = []
for i in range(len(un_splited_uni_name)):
    #print(un_splited_uni_name[i])
    a = un_splited_uni_name[i]
    b = a.split(' - ')[0]
    #print(b)
    uni_name_matrix.append(b)

#### We have to remove the () for some universities

In [170]:
def extract_info(uni_name):
    # url
    original_url = 'http://www.geonames.org/search.html?'
    payload={}
    payload['q']=uni_name
    payload['country']='CH'
    get_url = requests.get(original_url, params=payload, timeout=30)
    soup = BeautifulSoup(get_url.text,"lxml")
    if soup.select('table tr td span[class=latitude]') == [] and soup.select('table tr td span[class=longitude]')==[]:
        #print('The univeristy:', uni_name, 'is not found!')
        latitude = 0;longitude = 0;state = np.nan
    else:
        latitude = float(soup.select('table tr td span[class=latitude]')[0].text)
        longitude = float(soup.select('table tr td span[class=longitude]')[0].text)
        l2 = soup.select('div table tr td')
        state = (l2[3].contents)[1].split(', ')[1]
    return latitude,longitude,state

In [178]:
# define the latitude and longitude matrix
latitude_longitude_dic = {}
# store the unsucceed university name
unsucceed_university_name = []
for uni_name in uni_name_matrix:
    latitude,longitude,state = extract_info(uni_name)
    if latitude==0:
        unsucceed_university_name.append(uni_name)
    else:
        latitude_longitude_dic[uni_name] = [state,latitude,longitude]    

In [179]:
latitude_longitude_dic

{'ETH Zürich': ['Zurich', 47.3763034523898, 8.54804992675781],
 'Kantonsspital St. Gallen': ['Saint Gallen',
  47.42925492892253,
  9.388259837297506],
 'Universität Basel': ['Basel-City', 47.55831967091123, 7.584031820297241],
 'Universität Bern': ['Bern', 46.95095609536347, 7.4354831442083436],
 'Universität Zürich': ['Zurich', 47.37092, 8.53434],
 'Université de Fribourg': ['Fribourg', 46.80683246786493, 7.153167516669125],
 'Université de Lausanne': ['Vaud', 46.52376278378653, 6.5840935707092285],
 'Université de Neuchâtel': ['Neuchâtel',
  46.993854489413216,
  6.937892486542709]}

In [180]:
unsucceed_university_name

['Nicht zuteilbar',
 'Université de Genève',
 'NPO (Biblioth., Museen, Verwalt.)',
 'Eidg. Forschungsanstalt für Wald,Schnee,Land',
 'Inst. de Hautes Etudes Internat. et du Dév',
 'Universität St. Gallen',
 'Weitere Institute',
 'Firmen/Privatwirtschaft',
 'Pädagogische Hochschule Graubünden',
 'EPF Lausanne',
 'Pädagogische Hochschule Zürich',
 'Universität Luzern',
 'Schweiz. Institut für Kunstwissenschaft',
 'SUP della Svizzera italiana',
 'HES de Suisse occidentale',
 'Robert Walser-Stiftung Bern',
 'Paul Scherrer Institut',
 'Pädagogische Hochschule St. Gallen',
 'Eidg. Anstalt für Wasserversorgung',
 'Eidg. Material und Prüfungsanstalt',
 'Physikal.-Meteorolog. Observatorium Davos',
 'Berner Fachhochschule',
 'Swiss Center for Electronics and Microtech.',
 'Weitere Spitäler',
 'AO Research Institute',
 'Allergie- und Asthmaforschung',
 'Forschungsinstitut für biologischen Landbau',
 'Friedrich Miescher Institute',
 'Forschungsanstalten Agroscope',
 'Ente Ospedaliero Cantonale',
 

In [35]:
# url
original_url = 'http://www.geonames.org/search.html?'
payload = {'q':'Université de Lausanne','country':'CH'}
get_url = requests.get(original_url, params=payload, timeout=30)
soup = BeautifulSoup(get_url.text,"lxml")

In [76]:
l = soup.select('table tr td span')
l

[<span id="topmenulogin">
 <a href="http://www.geonames.org/login" title="login or create new user account">login</a>
 </span>,
 <span class="geo" style="display:none;"><span class="latitude">46.52376278378653</span><span class="longitude">6.5840935707092285</span></span>,
 <span class="latitude">46.52376278378653</span>,
 <span class="longitude">6.5840935707092285</span>]

In [98]:
latitude = float(soup.select('table tr td span[class=latitude]')[0].text)
latitude

46.52376278378653

In [99]:
longitude = float(soup.select('table tr td span[class=longitude]')[0].text)
longitude

6.5840935707092285

In [90]:
l = soup.select('div table tr td')
(l[3].contents)[1].split(', ')[1]

'Vaud'

In [100]:
l[3].contents

[<a href="/countries/CH/switzerland.html">Switzerland</a>,
 ', Vaud',
 <br/>,
 <small>Ouest Lausannois District &gt; Chavannes-près-Renens</small>,
 '\n']

In [97]:
if soup.select('table[class=restable]') is []:
    print (1)
else:
    print (0)

0
